In [ ]:
import worldcat_expanding
import csv
import json
import lxml.etree as ET
import requests

In [ ]:
with open('output/Summerbookstore2017.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    bookstore_isbns = [row[7].replace('-', '')
                       for row in spamreader
                       if 'ISBN' not in row[7]
                       and 'E BOOK' not in row[7]]

In [ ]:
Spring3_etextbooks = ['9780123786326',
                          '9780080470030',
                          '9780300158373',
                          '9780300180213',
                          '9780415921428',
                          '9780765626400',
                          '9780415943918',
                          '9781136611230',
                          '9780415535878',
                          '9780521176149',
                          '9781107607453',
                          '9780521423359',
                          '9780511780882',
                          '9780521542449',
                          '9780511164798',
                          '9780822957539',
                          '9781134711345',]

In [ ]:
spring_ebook_to_worldcat_api = dict()

for isbn in sorted(Spring3_etextbooks):
    isbns, marcrecords = worldcat_expanding.main(isbn)
    if isbns:
        spring_ebook_to_worldcat_api[isbn] = isbns

In [ ]:
spring_ebook_to_xisbn = dict()

for isbn in sorted(Spring3_etextbooks):
    url = 'http://xisbn.worldcat.org/webservices/xid/isbn/{}?method=getEditions&format=json&ai=mike.waugh'.format(isbn)
    response = requests.get(url)
    raw_json = response.text
    parsed_json = json.loads(raw_json)
    isbns = set()
    if parsed_json['stat'] == 'ok':
        for item in parsed_json['list']:
            for _, sublist in item.items():
                for i in sublist:
                    isbns.add(i)
    spring_ebook_to_xisbn[isbn] = isbns

In [ ]:
spring_ebook_to_thingISBN = dict()

for isbn in sorted(Spring3_etextbooks):
    url = 'http://www.librarything.com/api/thingISBN/{}'.format(isbn)
    response = requests.get(url)
    response_etree = ET.fromstring(response.content)
    isbns = set()
    for child in response_etree:
        isbns.add(child.text)
    spring_ebook_to_thingISBN[isbn] = isbns

In [ ]:
for k,v in sorted(spring_ebook_to_xisbn.items()):
    print(k, v)
    print(v.intersection(spring_ebook_to_worldcat_api.get(k)))
    break
    


In [ ]:
worldcat_expanding.main('9780750677653')[0]

In [ ]:
worldcat_pw = worldcat_expanding.get_password()

def stretch_isbn_list(isbn):
    starting_isbns = {isbn, }
    while True:
        current_list = list(starting_isbns)    
        for uid in current_list:
            alternate_isbns, _ = worldcat_expanding.lookup_alternates(uid, worldcat_pw)
            starting_isbns.update(alternate_isbns)
        print(sorted(starting_isbns))
        print(sorted(set(current_list)))
        if set(current_list) == starting_isbns:
            return starting_isbns
    

In [ ]:
stretched_set = stretch_isbn_list(target_isbn)

In [ ]:
# map physical_isbns to ebook_isbns for the spring ebook set
# send each api a physical_isbn & expect the ebook_isbn in the return value


In [ ]:
physical_to_ebook = [('9780123786326', '9780123786333'),
                     ('')]